In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Experiments: Autologging

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10.10

## Overview

As part of the data science team, you want to try different modeling approaches during experimentation phase.To guarantee reproducibility, each approach has different parameters that you need to manually track This is a time consuming task. To address this challenge, Vertex AI SDK introduces autologging, a one-line code SDK capability which leverages MLflow to provide automatic metrics and parameters tracking associated with your  Vertex AI Experiments and experiment runs. Learn more about [Autologging data to an experiment run](https://cloud.google.com/vertex-ai/docs/experiments/autolog-data).

### Objective

In this tutorial, you learn how to use `Vertex AI Autologging`.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Experiments

The steps performed include:

- Enable autologging in the Vertex AI SDK.
- Train one scikit-learn model
- Train one LightGBM model
- Train three XGBoost models (setting different hyperparameters)
- See all the resulting experiment runs with the evaluation metrics (accuracy, precision, recall, and F1-score) and parameters autologged to Vertex AI Experiments without setting any experiment run.


### Dataset

The dataset is the [UCI Car Evaluation dataset](https://archive-beta.ics.uci.edu/dataset/19/car+evaluation), which is derived from simple hierarchical decision model and it contains attributions to predict car evaluation class.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Experiments
* Vertex AI Tensorboard
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [2]:
# Install the packages
USER = ""
! pip3 install {USER} --upgrade --q google-cloud-aiplatform tensorflow
! pip3 install {USER} --upgrade --q pandas scikit-learn matplotlib category_encoders xgboost==1.7.5 lightgbm==3.3.5 mlflow
! pip3 install {USER} --upgrade --q protobuf==3.20.3

### Colab only: Uncomment the following cell to restart the kernel.
Automatically restart kernel after installs so that your environment can access the new packages

In [3]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Setup

### Define env variables

In [4]:
# Set the project id
PROJECT_ID = "my-project-id"  # @param {type:"string"}
! gcloud config set project {PROJECT_ID}

# Set your region
REGION = "europe-west4"  # @param {type: "string"}

# Set a bucket to store intermediate artifacts
BUCKET_URI =f"gs://experiments-demo-{PROJECT_ID}"  # @param {type:"string"}

Updated property [core/project].


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [5]:
# ! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Upload dataset to BigQuery, load it to a pandas dataframe

1. Upload the [ UCI Car Evaluation dataset](https://archive-beta.ics.uci.edu/dataset/19/car+evaluation) (mentioned above) to a BigQuery table (**your turn**)
2. Upload data from this table to a pandas dataframe.

(you could also interact with the data in CSV format if you prefer, but then you'd need to do some minor changes to the code)

#### 2. Upload BigQuery table to a pandas dataframe

In [6]:
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client = bigquery.Client()

## JUST FOR COLAB
#from google.colab import data_table
#data_table.enable_dataframe_formatter()

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [7]:
%%bigquery cars_df --project $PROJECT_ID

SELECT * FROM `experiments_workshop.car_eval_preprocessed` # @param {type: "string"}

Query is running:   0%|          |

Downloading:   0%|          |

In [8]:
cars_df

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,0
1,vhigh,vhigh,2,2,med,low,0
2,vhigh,vhigh,2,2,big,low,0
3,vhigh,vhigh,2,4,small,low,0
4,vhigh,vhigh,2,4,med,low,0
...,...,...,...,...,...,...,...
1723,low,low,5more,4,med,high,1
1724,low,low,5more,4,big,high,1
1725,low,low,5more,more,small,high,1
1726,low,low,5more,more,med,high,1


### Import libraries

Import the Vertex AI SDK to log experiments in Vertex AI Experiments.

In [9]:
from google.cloud import aiplatform as vertex_ai

### Helper functions

To run experiments it is not uncommon to define experiment helpers, one per each modelling approach you plan to evaluate. Below you define the following experiment helpers:

*   `train_sklearn_model`: A helper function to train a Decision Tree model using Sklearn.
*   `train_xgboost_model`: A helper function to train a Decision Tree model using XGBoost.
*   `train_LGBM_model`: A helper function to train a Decision Tree model using LightGBM.


In [10]:
def set_seed(seed: int):
    """
    A function to set the seed for reproducibility.
    Args:
        seed: Seed to be set
    Returns:
        None
    """
    import random

    import numpy as np
    import tensorflow as tf

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
import pandas as pd


def train_xgboost_model(df: pd.DataFrame, test_size: int, max_depth: int, n_estimators: int,  enable_categorical: bool=True):
    """
    A function to train a Decision Tree model using XGboost.
    Args:
        df: Input data, which needs to be split into train and test
        test_size: Size of the test set
        max_depth: Maximum depth of the Decision Tree
        n_estimators: Number of trees
        enable_categorical: Whether to enable categorical features
    Returns:
        None
    """

    # Libraries
    
    from xgboost import XGBClassifier
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder


    # Convert categorical columns to numerical values
    for column in cars_df.select_dtypes(include='object').columns:
        label_encoder = LabelEncoder()
        cars_df[column] = label_encoder.fit_transform(cars_df[column])

    # Train, test split
    print("Generating train and test data...")
    x = cars_df[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    y = cars_df[["class"]]
    X_train, X_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, shuffle=True
    )

    # Train model
    print("Training model...")
    xgb_clf = XGBClassifier(
        max_depth=max_depth, n_estimators=n_estimators, objective="binary:logistic"
    )
    xgb_clf.fit(X_train, y_train)

    # Evaluate model
    print("Evaluating model...")
    y_pred = xgb_clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:", round(accuracy, 3))
    precision = metrics.precision_score(y_test, y_pred)
    print("Precision:", round(precision, 3))
    recall = metrics.recall_score(y_test, y_pred)
    print("Recall:", round(recall, 3))
    f1 = metrics.f1_score(y_test, y_pred)
    print("F1 score:", round(f1, 3))


def train_sklearn_model(df: pd.DataFrame, test_size: int, max_depth: int, enable_categorical: bool=True):
    """
    A function to train a Decision Tree model using scikit-learn.
    Args:
        df: Input data, which needs to be split into train and test
        test_size: Size of the test set
        max_depth: Maximum depth of the Decision Tree
        n_estimators: Number of trees
        enable_categorical: Whether to enable categorical features
    Returns:
        None
    """

    # Libraries
    from sklearn.tree import DecisionTreeClassifier
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    import matplotlib

    # Convert categorical columns to numerical values
    for column in cars_df.select_dtypes(include='object').columns:
        label_encoder = LabelEncoder()
        cars_df[column] = label_encoder.fit_transform(cars_df[column])

    # Train, test split
    print("Generating train and test data...")
    x = cars_df[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    y = cars_df[["class"]]
    X_train, X_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, shuffle=True
    )

    # Train model
    print("Training model...")
    dt_clf = DecisionTreeClassifier(max_depth=max_depth)
    dt_clf.fit(X_train, y_train)

    # Evaluate model
    print("Evaluating model...")
    y_pred = dt_clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:", round(accuracy, 3))
    precision = metrics.precision_score(y_test, y_pred)
    print("Precision:", round(precision, 3))
    recall = metrics.recall_score(y_test, y_pred)
    print("Recall:", round(recall, 3))
    f1 = metrics.f1_score(y_test, y_pred)
    print("F1 score:", round(f1, 3))
    

def train_lgbm_model(df, test_size=0.25, max_depth=3, n_estimators=100):
    """
    A function to train a Light GBM model.
    Args:
        df: Input data, which needs to be split into train and test
        test_size: Size of the test set
        max_depth: Maximum depth of the Decision Tree
        n_estimators: Number of trees
        enable_categorical: Whether to enable categorical features
    Returns:
        None
    """

    # Libraries
    import lightgbm as lgb
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder

    # Convert categorical columns to numerical values
    for column in cars_df.select_dtypes(include='object').columns:
        label_encoder = LabelEncoder()
        cars_df[column] = label_encoder.fit_transform(cars_df[column])

    # Train, test split
    print("Generating train and test data...")
    x = cars_df[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    y = cars_df[["class"]]
    X_train, X_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, shuffle=True
    )

    # Create a label encoder for the target variable
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)

    # Create a LightGBM model
    model = lgb.LGBMClassifier(objective='binary', max_depth=max_depth, n_estimators=n_estimators)

    # Train the model on the train set
    print("Training model...")
    model.fit(X_train, y_train)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:", round(accuracy, 3))
    precision = metrics.precision_score(y_test, y_pred)
    print("Precision:", round(precision, 3))
    recall = metrics.recall_score(y_test, y_pred)
    print("Recall:", round(recall, 3))
    f1 = metrics.f1_score(y_test, y_pred)
    print("F1 score:", round(f1, 3))

## Model experimentation using autologging with Vertex AI Experiments

Vertex AI Experiments Autologging allows you to run experiments and autologging parameters and metrics of different ML frameworks.

After initiating an Vertex AI Experiment, enable autologging using `vertex_ai.autolog()`.

There are two ways to use Autologging:

1.   *With automatic experiment run creation*
2.   *With user experiment run creation*

With *automatic experiment run creation*, you run an experiment. Vertex AI SDK automatically creates an experiment run by logging all paramenters and metrics in Vertex AI Experiments.

With *user experiment run creation*, you create an experiment using `vertex_ai.start_run(your-experiment-run-name)` and run the experiment. Then you get access to resulting paramentes and metrics after you end the experiment run with `vertex_ai.end_run()`


#### Create an experiment for tracking training parameters and metrics

To start, initiate an experiment using the `init()` method.

Because some model types like TensorFlow result in autologging time series metrics, you need to create a TensorBoard instance.

To create a TensorBoard instance, you can use `vertex_ai.Tensorboard.create()`.


 ⚠⚠⚠ **Warning** Currently, Vertex AI TensorBoard charges a monthly fee of 300 USD per unique active user. Beginning in August 2023, Vertex AI TensorBoard will change from a per-user monthly license of 300 USD per month to 10 USD per GiB per month for storage of logs and metrics. This means there are no more subscription fees, and you will only pay for the storage you've used. See the [Vertex AI TensorBoard: Delete Outdated TensorBoard Experiments tutorial](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb) for how to manage storage. More info on Tensorboard [here](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview).


In [11]:
AUTOLOGGED_EXPERIMENT_NAME = "my-experiment"  # @param {type:"string"}

In [12]:
EXPERIMENT_TENSORBOARD = vertex_ai.Tensorboard.create()

Creating Tensorboard
Create Tensorboard backing LRO: projects/1080686785400/locations/us-central1/tensorboards/3902149174790979584/operations/28639185527111680
Tensorboard created. Resource name: projects/1080686785400/locations/us-central1/tensorboards/3902149174790979584
To use this Tensorboard in another session:
tb = aiplatform.Tensorboard('projects/1080686785400/locations/us-central1/tensorboards/3902149174790979584')


In [13]:
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
    experiment=AUTOLOGGED_EXPERIMENT_NAME,
    experiment_tensorboard=EXPERIMENT_TENSORBOARD,
    experiment_description="autolog-experiment-with-automatic-run",
)

#### Autologging an experiment with automatic experiment run creation

In this section, Vertex AI SDK automatically creates an experiment run for you by logging all paramenters and training and post-training metrics in Vertex AI Experiments.


##### Enable autologging

First, enable autologging using `vertex_ai.autolog()` method.

After calling `vertex_ai.autolog()`, any metrics and parameters from
model training calls with supported ML frameworks will be automatically
logged to Vertex Experiments.

In [14]:
vertex_ai.autolog()

#### Experiment 1 - Sklearn model

Next, define your baseline model by running a Sklearn model experiment.

In [15]:
train_sklearn_model(cars_df, test_size=0.2, max_depth=5)

Generating train and test data...
Training model...
Associating projects/1080686785400/locations/europe-west4/metadataStores/default/contexts/my-experiment-sklearn-2023-07-27-18-57-13-0f510 to Experiment: my-experiment
Evaluating model...
Accuracy: 0.948
Precision: 0.71
Recall: 0.71
F1 score: 0.71


##### Get the experiment results

Then, use the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

Notice how all paramenters and metrics are logged in Vertex AI Experiments.

In particular, the `run_name` has been automatically assigned and the evaluation metrics you defined have been logged too.

In [16]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df = experiment_df.T
experiment_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
experiment_name,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,...,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment,my-experiment
run_name,sklearn-2023-07-27-18-57-13-0f510,lightgbm-2023-07-27-18-43-42-6a461,sklearn-2023-07-27-18-35-28-9337e,sklearn-2023-07-26-12-59-42-89c02,sklearn-2023-07-26-12-54-00-35fc5,sklearn-2023-07-26-12-34-34-f191b,sklearn-2023-07-26-12-34-06-f5757,sklearn-2023-07-26-12-33-03-d4572,sklearn-2023-07-26-12-31-13-0eb6a,sklearn-2023-07-26-12-28-01-05c79,...,xgboost-2023-07-26-12-10-41-e7f18,lightgbm-2023-07-26-12-10-31-1cbc7,sklearn-2023-07-26-12-10-02-e6da6,sklearn-2023-07-26-12-04-27-3195e,sklearn-2023-07-26-12-02-34-99200,xgboost-2023-07-26-11-40-22-71be7,xgboost-2023-07-26-11-40-13-7e36e,xgboost-2023-07-26-11-39-58-cd7a4,lightgbm-2023-07-26-11-35-23-83821,sklearn-2023-07-26-11-35-08-cbf96
run_type,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,...,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun,system.ExperimentRun
state,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,...,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE,COMPLETE
param.class_weight,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
metric.recall_score_X_test,NaN,0.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.928571,0.75,NaN,NaN,NaN,NaN,NaN,NaN,0.75,NaN
metric.precision_score_x_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.941176,0.925926,0.764706,NaN,NaN
metric.recall_score_x_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.961538,0.928571,NaN,NaN
metric.accuracy_score_x_test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.99711,0.991329,0.971098,NaN,NaN


#### Experiment 2 - LGBM model.

In [17]:
train_lgbm_model(cars_df, test_size=0.2, max_depth=3, n_estimators=50)

Generating train and test data...
Training model...


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Associating projects/1080686785400/locations/europe-west4/metadataStores/default/contexts/my-experiment-lightgbm-2023-07-27-18-58-13-a66d9 to Experiment: my-experiment
Accuracy: 0.957
Precision: 0.842
Recall: 0.78
F1 score: 0.81


#### Experiment 3 - XGBoost model

Let's train 3 models, changing the parameters max_depth and n_estimators.

In [18]:
train_xgboost_model(cars_df, test_size=0.2, max_depth=3, n_estimators=50)

Generating train and test data...
Training model...
Associating projects/1080686785400/locations/europe-west4/metadataStores/default/contexts/my-experiment-xgboost-2023-07-27-18-58-20-feb4e to Experiment: my-experiment
Evaluating model...
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0


In [19]:
train_xgboost_model(cars_df, test_size=0.2, max_depth=5, n_estimators=100)

Generating train and test data...
Training model...
Associating projects/1080686785400/locations/europe-west4/metadataStores/default/contexts/my-experiment-xgboost-2023-07-27-18-58-28-36f71 to Experiment: my-experiment
Evaluating model...
Accuracy: 0.988
Precision: 1.0
Recall: 0.833
F1 score: 0.909


In [20]:
train_xgboost_model(cars_df, test_size=0.2, max_depth=10, n_estimators=200)

Generating train and test data...
Training model...
Associating projects/1080686785400/locations/europe-west4/metadataStores/default/contexts/my-experiment-xgboost-2023-07-27-18-58-37-b317a to Experiment: my-experiment
Evaluating model...
Accuracy: 0.994
Precision: 1.0
Recall: 0.913
F1 score: 0.955


#### Compare experiment results

In [ ]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df.T